# Thought Organizer

In [15]:
import configparser
from openai import OpenAI
import pandas as pd
from typing import List

## Tags

### Helpers

In [19]:
def load_csv(filename: str) -> pd.DataFrame:
    return pd.read_csv(filename)

def generate_tags(
    client: OpenAI,
    input_csv: pd.DataFrame,
) -> pd.DataFrame:

    MAX_NO_TAGS = 3
    TEXT_COLUMNS = "front,back"
    NEW_COLUMN = "new_tags"
    prompt = f"""Below is a csv where a row represents the front and back of a virtual quiz card.
             Each card has a set of tags associated with it, and some other metadata.
             
             Please revise the existing tags, based on the text in the columns {TEXT_COLUMNS}.
             
             Strive for generic tags and low total of unique tags.
             Each card can have a maximum of {MAX_NO_TAGS} tags.
             Merge similar tags.
             
             The output should be the same csv, with an additional column '{NEW_COLUMN}'.
             '{NEW_COLUMN}' should contain your suggestions.
             The response should not contain any text except the csv.

             ---

             {input_csv.to_csv(index=False)}
             """
    
    MODEL = "gpt-3.5-turbo"
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )

    MESSAGE = 0
    return response.choices[MESSAGE]

# Function to process CSV and add tags
def process_csv(input_filename: str, output_filename: str) -> None:
    df = load_csv(input_filename)    
    df_with_tags = generate_tags(df)
    # df_with_tags.to_csv(output_filename, index=False)
    # print(f"File saved to {output_filename}")

## Main

In [25]:
config = configparser.ConfigParser()
config.read("config.ini")

client = OpenAI(
    api_key=config["OpenAI"]["API_KEY"],
)

input_filename: str = config["Files"]["INPUT_FILE"]

df = load_csv(input_filename)